In [1]:
import scanpy as sc
import numpy as np
import pickle
import umap
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

/home/shawn/miniconda3/envs/msse-python/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load adata
#adata = sc.read_h5ad('TS.h5ad')
#pickle.dump(adata.X, open('TS_X.pickle', 'wb'))
#pickle.dump(adata.obs['free_annotation'], open('cell_labels.pickle', 'wb'))

In [3]:
f = open('TS_X.pickle', 'rb')
matrix = pickle.load(f)

In [4]:
f.close()

In [8]:
index = np.arange(100000)
np.random.shuffle(index)
matrix = matrix[index, :]

In [9]:
import gc
gc.collect()

474

In [10]:
matrix = matrix.todense()

In [11]:
matrix = np.asarray(matrix)

In [ ]:
import struct
with open('matrix.dat', 'wb') as f:
    for row in matrix:
        # IO is slow but I think this helps by putting like 58,000 things in one syscall
        f.write(struct.pack('f'*matrix.shape[1], *list(row)))

In [13]:
reducer = umap.UMAP()

In [14]:
# Double check that it really is scaled
for i in range(matrix.shape[0]):
    if np.max(matrix[i,:]) > 10:
        print(np.max(matrix[i,:]))

In [ ]:
embedding = reducer.fit_transform(matrix)
embedding.shape


/home/shawn/miniconda3/envs/msse-python/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [7]:
adata.obsm['X_umap']

array([[ 9.770117 ,  3.8183944],
       [ 9.286816 ,  4.2755036],
       [17.679771 ,  9.184891 ],
       ...,
       [ 4.9387674,  8.494324 ],
       [ 4.680503 ,  8.350122 ],
       [ 4.67996  ,  8.363431 ]], dtype=float32)

In [10]:
um = umap.UMAP()
um.embeddings_ = adata.obsm['X_umap']

In [11]:
um.embeddings_

array([[ 9.770117 ,  3.8183944],
       [ 9.286816 ,  4.2755036],
       [17.679771 ,  9.184891 ],
       ...,
       [ 4.9387674,  8.494324 ],
       [ 4.680503 ,  8.350122 ],
       [ 4.67996  ,  8.363431 ]], dtype=float32)

In [21]:
X_test = np.array(adata.X[0:5].todense())

In [22]:
new_embedding = um.transform(X_test)


AttributeError: 'UMAP' object has no attribute '_raw_data'

In [ ]:

# file = open('ts_umap.pickle', 'rb')
# ts_umap = pickle.load(file)

# creates a new embedding based on the ts embeddings
new_embedding = ts_umap.transform(X_test)

# now we need to use knn, SVC or some other unsupervised classifier to check closeness of points to original ts_umap embedding
svc = SVC().fit(new_embedding, ts_umap)
knn = KNeighborsClassifier().fit(new_embedding, ts_umap)

# Regardless of which one we choose we need to map the label numbers to the true labels from ts_umap, then map those to the
# cell names


In [ ]:
# Make a pickle of the ts dataset UMAP by loading entire dataset, performing umap and constructing a sklearn umap() object
# Then pickle this object and while you're at it store a record of the ts_labels with barcodes
# Don't think its a bad idea to just store this in a sqlite db as well now, although the pickle will just be
# stored on the file system

ts_umap = umap.UMAP()
ts_umap.embeddings_ = # insert code to copy the tabula sapiens embeddings into here
pickle.dump(ts_umap, open('ts_umap.pickle', 'wb'))